In [2]:
import pandas as pd
import os
import time
import csv
import re
import codecs
import requests
from bs4 import BeautifulSoup
 
import datetime
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, \
        TimeoutException, ElementNotInteractableException, ElementClickInterceptedException

import warnings
warnings.filterwarnings("ignore")

In [ ]:
url = f'https://www.goodreads.com/list/show/7.Best_Books_of_the_21st_Century?page=1'

response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
soup

In [ ]:
page1 = soup.find_all('a', class_='bookTitle')

In [ ]:
page1

In [ ]:
test = page1[0]['href']

In [ ]:
test

In [ ]:
re.search(r'\d.*', test)

In [ ]:
len(page1)

In [ ]:
book_ids = []

for i in range(0, len(page1)):
    
    href = page1[i]['href']
               
    book_ids.append(re.findall(r'\d.*', href))

In [ ]:
book_ids

In [ ]:
books = []

for i in range(1, 51):
    
    url = f'https://www.goodreads.com/list/show/7.Best_Books_of_the_21st_Century?page={i}'

    response = requests.get(url)
    
    soup = BeautifulSoup(response.content, 'html.parser')

    page = soup.find_all('a', class_='bookTitle')
    
    print(response.status_code)
    
    for i in range(0, len(page)):
    
        href = page[i]['href']
               
        books.append(re.findall(r'\d.*', href))
        
    time.sleep(3)

In [ ]:
books

In [ ]:
books_5000_list = []

#url = 'https://www.goodreads.com/book/show/' + book_id


for i in range(len(books)):
    
    book = books[i][0]
    
    books_5000_list.append(book)

In [ ]:
books_5000_list

In [ ]:
len(books_5000_list)

In [ ]:
books_2000 = books[0:2000]

In [ ]:
len(books_2000)

In [ ]:
books_2000

In [ ]:
books_2000_list = []

#url = 'https://www.goodreads.com/book/show/' + book_id


for i in range(len(books_2000)):
    
    book = books[i][0]
    
    books_2000_list.append(book)

In [ ]:
len(books_2000_list)

In [ ]:
with open("5000_book_ids.txt", "w") as output:
#    output.write(str(book_list))
    for i in books_5000_list:
        output.write(i + '\n')

In [ ]:
with open("2000_book_ids.txt", "w") as output:
#    output.write(str(book_list))
    for i in books_2000_list:
        output.write(i + '\n')

## running through gathering all of the data for one book

In [ ]:
base = 'https://www.goodreads.com/book/show/'

df = pd.read_csv('2000_book_ids.txt', header= None, names = ['book_tag'])

practice = base +  df['book_tag'][0]
practice

In [ ]:
response = requests.get(practice)

soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
collect = pd.DataFrame(columns = ['title','summary','year_published','author','review_count',
                                  'number_of_ratings','length','genre','rating','list','reviews'])

## getting the title

In [ ]:
soup.find('h1', attrs={'class': 'Text Text__title1'})

In [ ]:
title_tag = soup.find('h1', attrs={'class': 'Text Text__title1'})

In [ ]:
title_tag['aria-label'].split(': ')[1]

In [ ]:
title = title_tag['aria-label'].split(': ')[1]

In [ ]:
collect.loc[0, 'title'] = title

## getting the author

In [ ]:
soup.find('span', class_ = "ContributorLink__name")

In [ ]:
soup.find('span', class_ = "ContributorLink__name").text

In [ ]:
author = soup.find('span', class_ = "ContributorLink__name").text

In [ ]:
collect.loc[0, 'author'] = author

## average rating

In [ ]:
soup.find_all('div', class_= 'RatingStatistics__rating')

In [ ]:
soup.find_all('div', class_= 'RatingStatistics__rating')[0].text

In [ ]:
rating = soup.find_all('div', class_= 'RatingStatistics__rating')[0].text

In [ ]:
collect.loc[0, 'rating'] = rating

## num of reviews and ratings

In [ ]:
soup.find_all('div', class_ = "RatingStatistics__meta")

In [ ]:
stats = soup.find_all('div', class_ = "RatingStatistics__meta")

In [ ]:
rating_span = soup.find('span', {'data-testid': 'ratingsCount'})
review_span = soup.find('span', {'data-testid': 'reviewsCount'})

In [ ]:
rating_text = rating_span.text.strip()
review_text = review_span.text.strip()

rating_text, review_text

In [ ]:
rating_text = re.sub('[^0-9]', '', rating_text)
review_text = re.sub('[^0-9]', '', review_text)
rating_text , review_text

In [ ]:
collect.loc[0, 'review_count'] = review_text

In [ ]:
collect.loc[0, 'number_of_ratings'] = rating_text

In [ ]:
collect

In [ ]:
#collect = collect.drop(columns='rating_text')

## summary

In [ ]:
soup.find_all('span', class_ = "Formatted")

In [ ]:
summary = soup.find_all('span', class_ = "Formatted")[0].text

In [ ]:
collect.loc[0, 'summary'] = summary

In [ ]:
collect

## genre

In [ ]:
soup.find('a', class_='Button Button--tag-inline Button--small')

In [ ]:
soup.find('a', {'class': 'Button Button--tag-inline Button--small'}).get_text()

In [ ]:
genre = soup.find('a', {'class': 'Button Button--tag-inline Button--small'}).get_text()

In [ ]:
genre

In [ ]:
collect.loc[0, 'genre'] = genre

## page count

In [ ]:
soup.find('p').text

In [ ]:
page = soup.find('p').text

In [ ]:
page = re.sub('[^0-9]', '', page)

In [ ]:
collect.loc[0, 'length'] = page

In [ ]:
collect

## publishing year

In [ ]:
(soup.find_all('p')[1].text)

In [ ]:
year = (soup.find_all('p')[1].text)

In [ ]:
year = year[-4:]

In [ ]:
year

In [ ]:
collect.loc[0, 'year_published'] = year

In [ ]:
collect

## reviews

In [ ]:
soup.find_all('section',class_ = "ReviewText__content")

In [ ]:
reviews = soup.find_all('section',class_ = "ReviewText__content")

In [ ]:
ls_reviews = []
for i in range (0,len (reviews)):
    rev = soup.find_all('section',class_ = "ReviewText__content")[i].text
    ls_reviews.append(rev)

In [ ]:
for i in range (0,len(reviews)):
    
    print(ls_reviews[i])
    print('\n')
    print('-------------------------')

In [ ]:
collect.loc[0, 'reviews'] = ls_reviews

In [ ]:
collect = collect.drop(columns='list')

In [ ]:
collect

In [ ]:
df

## series

In [ ]:
soup

In [ ]:
#soup.find('a', {'class': 'Button Button--tag-inline Button--small'}).get_text()

In [ ]:
soup.find_all('div', class_='WorkDetails')

In [ ]:
soup.find('body', {'class': 'DescListItem'})

In [ ]:
soup.find('a')['href']

In [ ]:
soup.find('dd')

In [ ]:
title_tag

In [14]:
df = pd.read_csv('final_links.txt', header=None)

In [15]:
df.head()

,0
0,https://www.goodreads.com/book/show/257233.Rai...
1,https://www.goodreads.com/book/show/205621.Pil...
2,https://www.goodreads.com/book/show/2398522.An...
3,https://www.goodreads.com/book/show/20497230-t...
4,https://www.goodreads.com/book/show/778667.The...


In [16]:
df = df.rename(columns={0:'book_tag'})

In [17]:
df

,book_tag
0,https://www.goodreads.com/book/show/257233.Rai...
1,https://www.goodreads.com/book/show/205621.Pil...
2,https://www.goodreads.com/book/show/2398522.An...
3,https://www.goodreads.com/book/show/20497230-t...
4,https://www.goodreads.com/book/show/778667.The...
...,...
4900,https://www.goodreads.com/book/show/24694140-a...
4901,https://www.goodreads.com/book/show/51286105-r...
4902,https://www.goodreads.com/book/show/62360044-s...
4903,https://www.goodreads.com/book/show/51853748-b...


In [21]:
# df = pd.read_csv('fiction-and-non-fiction-top-best-sellers.csv', header= None)

base = 'https://www.goodreads.com/book/show/'

collect = pd.DataFrame(columns = ['title','summary'])

# options = Options()
# options.headless = True
# service = Service('/path/to/chromedriver') # replace with the path to your chromedriver executable
driver = webdriver.Firefox()

counter = 0

for i, item in enumerate(df['book_tag'][351:]):
    
    url = item
    driver.get(url)
    # Wait for the page to load
    wait = WebDriverWait(driver, 30)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'Text__title1')))

    # Convert the page source to a BeautifulSoup object
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    print(driver.execute_script('return document.readyState'))  # should print 'complete' 
    print(f'URL:              {url}')
    # print(f'Request Response: {response.status_code}')
    print(f'Request Number:   {counter+1}')
    
    #--------title-------------------------------------------
    
    # save to Variable
    title_tag = soup.find('h1', attrs={'class': 'Text Text__title1'}) #.text
    # looking at the h1 tag aria.        
    title = title_tag['aria-label'].split(': ')[1]
    print(f'Book Title: {title}')
    
    #----author-----------------------------------------------
    
    # find where author is stored
    # soup.find('span', class_ = "ContributorLink__name")
    # save the text to a variable
    # auth = soup.find('span', class_ = "ContributorLink__name").text
    # print(f'Book Author: {auth}')
    
    #----rating-----------------------------------------------
    
    # 
    # rating = soup.find_all('div', class_= 'RatingStatistics__rating')[0].text
    # print(f'Overall Rating: {rating}')
    
    #----genre------------------------------------------------
    
    # try:
    #     genre = soup.find('a', {'class': 'Button Button--tag-inline Button--small'}).get_text()
    #     print(f'Genre: {genre}')
    # except:
    #     continue
    
    #----rating and review counts------------------------------
    
    # stats = soup.find_all('div', class_ = "RatingStatistics__meta")
    
    # rating_span = soup.find('span', {'data-testid': 'ratingsCount'})
    # review_span = soup.find('span', {'data-testid': 'reviewsCount'})
    
    # rating_text = rating_span.text.strip()
    # review_text = review_span.text.strip()
    
    # rating_text = re.sub('[^0-9]', '', rating_text)
    # review_text = re.sub('[^0-9]', '', review_text)
    # print(f'Number of Ratings: {rating_text}')
    # print(f'Number of Reviews: {review_text}')
         
    #----summary------------------------------
    
    summary = soup.find_all('span', class_ = "Formatted")[0].text
           
    #----# of pages------------------------------
    
    # page = soup.find('p').text
    # page = re.sub('[^0-9]', '', page)
    # print(f'Number of Pages: {page}')
          
    #--------year--------------------------
    
    # year = (soup.find_all('p')[1].text)
    # year = year[-4:]
    # print(f'Year Published: {year}')
    
    #--------reviews--------------------------

    # reviews = soup.find_all('section',class_ = "ReviewText__content")
          
    # ls_reviews = []
    # for i in range (0,len (reviews)):
    #    rev = soup.find_all('section',class_ = "ReviewText__content")[i].text
    #    ls_reviews.append(rev)
    
    # print('---------------------------------------------')
        
    collect.loc[counter, ['title','summary']] = title, summary
    
    counter += 1 
    

complete
URL:              https://www.goodreads.com/book/show/74033.Innocent_in_Death?from_search=true&from_srp=true&qid=yvJfCnFAq7&rank=1
Request Number:   1
Book Title: Innocent in Death
complete
URL:              https://www.goodreads.com/book/show/9420.Shopaholic_Baby?from_search=true&from_srp=true&qid=5NLG0G7enr&rank=1
Request Number:   2
Book Title: Shopaholic & Baby
complete
URL:              https://www.goodreads.com/book/show/49676332-study-guide?from_search=true&from_srp=true&qid=AUVmkIGOP4&rank=1
Request Number:   3
Book Title: Study Guide
complete
URL:              https://www.goodreads.com/book/show/33281554-mary-higgins-clark-collection-6-books-bundle?from_search=true&from_srp=true&qid=Uft7bcg8b4&rank=1
Request Number:   4
Book Title: Mary Higgins Clark Collection 6 Books Bundle
complete
URL:              https://www.goodreads.com/book/show/597790.The_Children_of_H_rin?from_search=true&from_srp=true&qid=IhhhCpQY1I&rank=1
Request Number:   5
Book Title: The Children of Hú

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


In [19]:
p1 = collect.copy()

In [20]:
p1

,title,summary
0,Raintree County,"Throughout a single day in 1892, John Shawness..."
1,Pilgrim's Inn,"After WW II, Lucilla Eliot's soldier son Georg..."
2,An A.J. Cronin Trilogy,Omnibus edition of two unabridged novels: The ...
3,The Naked and the Dead by Norman Mailer Lesson...,The Naked and the Dead lesson plan contains a ...
4,The Big Fisherman,"It was a calm, early summer noon in the southe..."
...,...,...
346,Lisey's Story,"Lisey Debusher Landon lost her husband, Scott,..."
347,Alex Cross Series James Patterson Collection 7...,Alex Cross Series James Patterson Collection 7...
348,Study Guide,"SuperSummary, a modern alternative to SparkNot..."
349,Plum Lovin',Mysterious men have a way of showing up in Ste...


In [ ]:
#df1 = df1.append(collect)

In [ ]:
#df = df[~df['book_tag'].isna()]

In [ ]:
#df1 = df1.drop_duplicates(subset='title')

In [ ]:
#df = pd.read_csv('almost_there.csv')

In [ ]:
#df = df.append(df1)

In [ ]:
#df.to_csv('almost_there.csv')

## testing beautiful soup

In [ ]:
url = 'https://www.goodreads.com/book/show/11735983-insurgent'

response = requests.get(url)

response

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
title_tag = soup.find('h1', attrs={'class': 'Text Text__title1'}).text

In [ ]:
title_tag

In [ ]:
    # looking at the h1 tag aria.
title = title_tag['aria-label'].split(': ')[1]

In [ ]:
title_tag.text

In [ ]:
title

In [ ]:
title_tag = soup.find('h1', attrs={'class': 'Text Text__title1'})

In [ ]:
df = pd.read_csv('2000_book_ids.txt')

In [ ]:
df

## selenium practice

In [ ]:
from selenium import webdriver
from parsel import Selector
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver = webdriver.Chrome()

In [ ]:
url = 'http://the-internet.herokuapp.com/login'

driver.get(url)

driver.find_element(By.XPATH, "//*[@id='username']").send_keys('tomsmith')
driver.find_element(By.XPATH, "//*[@id='password']").send_keys('SuperSecretPassword!')
driver.find_element(By.XPATH, '//*[@id="login"]/button').click()

In [ ]:
# old syntax for the above
# driver.find_element_by_xpath('//*[@id="username"]').send_keys('tomsmith')
# driver.find_element_by_xpath('//*[@id="password"]').send_keys('SuperSecretPassword!')
# driver.find_element_by_xpath('//*[@id="login"]/button').click()

In [ ]:
url = 'https://the-internet.herokuapp.com/dynamic_loading/2'

driver.get(url)

driver.find_element(By.XPATH, '//*[@id="start"]/button').click()
#implicit wait
driver.implicitly_wait(10)
text = driver.find_element(By.XPATH, "//*[@id='finish']/h4").text

print(text)

In [ ]:
driver.quit()

In [ ]:
url = 'https://www.youtube.com/@TinaHuang1/videos'

driver = webdriver.Chrome()

driver.get(url)

In [ ]:
# configure webdriver
options = Options()
options.headless = False  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
...
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    # this will disable image loading
    "prefs", {"profile.managed_default_content_settings.images": 2}
)
...
driver = webdriver.Chrome(options=options, chrome_options=chrome_options)
driver.get("https://www.twitch.tv/directory/game/Art")
# wait for page to load
element = WebDriverWait(driver=driver, timeout=5).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-target=directory-first-item]'))
)
print(driver.page_source)

In [ ]:
sel = Selector(text=driver.page_source)
parsed = []
for item in sel.xpath("//div[contains(@class,'tw-tower')]/div[@data-target]"):
    parsed.append({
        'title': item.css('h3::text').get(),
        'url': item.css('.tw-link::attr(href)').get(),
        'username': item.css('.tw-link::text').get(),
        'tags': item.css('.tw-tag ::text').getall(),
        'viewers': ''.join(item.css('.tw-media-card-stat::text').re(r'(\d+)')),
    })

In [ ]:
parsed

In [ ]:
    reviews = soup.find_all('section',class_ = "ReviewText__content")
          
    ls_reviews = []
    for i in range (0,len (reviews)):
        rev = soup.find_all('section',class_ = "ReviewText__content")[i].text
        ls_reviews.append(rev)

In [ ]:
df

In [ ]:
collectS = pd.DataFrame(columns = ['title','summary','year_published','author','review_count','number_of_ratings','length','genre','rating','reviews'])


options = Options()
options.headless = True
service = Service('/path/to/chromedriver') # replace with the path to your chromedriver executable
driver = webdriver.Chrome(service=service, options=options)

for i, item in enumerate(df['book_tag']):

    print(counter)

    url = base + item
    driver.get(url)
    # Wait for the page to load
    wait = WebDriverWait(driver, 30)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'Text__title1')))

    # Convert the page source to a BeautifulSoup object
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    print(driver.execute_script('return document.readyState'))  # should print 'complete'

    #--------title-------------------------------------------

    # save to Variable
    title_tag = soup.find('h1', attrs={'class': 'Text Text__title1'})
    # looking at the h1 tag aria.
    title = title_tag['aria-label'].split(': ')[1]
    print(title)

    #----author-----------------------------------------------

    # find where author is stored
    soup.find('span', class_="ContributorLink__name")
    # save the text to a variable
    auth = soup.find('span', class_="ContributorLink__name").text
    print(auth)

    #----rating-----------------------------------------------

    rating = soup.find_all('div', class_='RatingStatistics__rating')[0].text
    print(rating)

    #----rating and review counts------------------------------

    stats = soup.find_all('div', class_="RatingStatistics__meta")

    rating_span = soup.find('span', {'data-testid': 'ratingsCount'})
    review_span = soup.find('span', {'data-testid': 'reviewsCount'})

    rating_text = rating_span.text.strip()
    review_text = review_span.text.strip()

    rating_text = re.sub('[^0-9]', '', rating_text)
    review_text = re.sub('[^0-9]', '', review_text)
    print(f'{rating_text} , {review_text}')

    #----summary------------------------------
    summary = soup.find_all('span', class_="Formatted")[0].text

    #----# of pages------------------------------
    page = soup.find('p').text
    page = re.sub('[^0-9]', '', page)
    print(page)

    #--------year--------------------------
    year = (soup.find_all('p')[1].text)
    year = year[-4:]
    print(year)

    #--------genre--------------------------
    try:
        genre = soup.find('a', {'class': 'Button Button--tag-inline Button--small'}).get_text()
        print(genre)
    except:
        continue

    #--------reviews--------------------------
    # reviews = soup.find_all('section', class_="ReviewText__content")

    # ls_reviews = []
    # for i in range(0, len(reviews)):
    #    rev = reviews[i].text  # using `reviews` instead of `soup`
    #    ls_reviews.append(rev)


    print('\n')
    print(counter)
    
    collectS.loc[counter, ['title', 'summary', 'year_published', 'author', 'review_count', 'number_of_ratings', 'length', 'genre', 'rating', 'reviews']] = title, summary, year, auth, review_text, rating_text, page, genre, rating, ls_reviews
    counter = counter + 1 

## gathering the data into one file

In [ ]:
#df1 = pd.read_csv('first_207_in_books.csv')
#df2 = pd.read_csv('second_149_in_books.csv')
#df3 = pd.read_csv('third_45_in_books.csv')
#df4 = pd.read_csv('fourth_102_in_books.csv')
#df5 = pd.read_csv('fifth_375_in_books.csv')

In [ ]:
#df6 = pd.read_csv('sixth_355_in_books.csv')

In [ ]:
#df_1 = df_1.append(df1)

In [ ]:
#df_1 = df_1.append(df2)

In [ ]:
#df_1 = df_1.append(df3)

In [ ]:
#df_1 = df_1.append(df4)

In [ ]:
#df_1 = df_1.append(df5)

In [ ]:
#df_1 = df_1.append(df6)

In [ ]:
#df_1 = df_1.drop(columns='Unnamed: 0')

In [ ]:
#df_1['title'].nunique()

In [ ]:
#df1 = df_1.drop_duplicates(subset='title')

In [ ]:
#df1

In [ ]:
#df1 = df1.reset_index()

In [ ]:
#df1 = df1.drop(columns='index')

In [ ]:
#lst = df1['genre'].value_counts().tail(65).index.to_list()

In [ ]:
#df1 = df1[~df1['genre'].isin(lst)]

In [ ]:
#df1

In [ ]:
#df1.to_csv('almost_there.csv')

In [ ]:
df = df.drop(columns='Unnamed: 0')

In [ ]:
df.to_csv('almost_there.csv')

## Creating a useable function from a combo of the above loop and Manny's selenium function

### But first, creating the function to get the urls

In [ ]:
def get_book_urls(filename, rng=15):
    '''
    This function will take in an optional range and scrape and then format the url of a
    books page to be input into a file to use for further scraping. Needs additional work to get it to work in a more universal way, 
    but functions as is. Need to edit function url for each different web page currently.
    '''
    books = []
    # initial loop to gather the data from Goodreads
    for i in range(1, rng):
        # The url where the books are gathered. This is the part that needs tweaking so that the function can just take in a url. 
        # Need to research how to format the string properly.
        url = f'https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page={i}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')  
        page = soup.find_all('a', class_='bookTitle')
        # Checking the status code as function runs
        print(response.status_code)
        # Appending the gathered data into a list of lists
        for i in range(0, len(page)):
            # Referencing the specific section of the hmtl to find the url
            href = page[i]['href']
            # Appending the url onto the list after shaving off the unneeded part with Regex   
            books.append(re.findall(r'\d.*', href))
            # books.append(href)
        # A short sleep timer so Goodreads doesn't get upset
        time.sleep(3)        
    books_list = []
    #url = 'https://www.goodreads.com/book/show/' + book_id
    # Short loop to turn the above created list of lists into a list for better formatting
    for i in range(len(books)):
        book = books[i][0]
        books_list.append(book)
    # Creating/Updating a file formatted properly for use later    
    with open(filename, "w") as output:
    # output.write(str(books_list))
        for i in books_list:
            output.write(i + '\n')

In [ ]:
get_book_urls('book_urls.txt', 5)

### From here, functionalizing each piece of info needed to be scraped

In [ ]:
def get_titles(df, link_col):
    
    options = Options()
    options.headless = True
    service = Service('/path/to/chromedriver') # replace with the path to your chromedriver executable
    driver = webdriver.Chrome(service=service, options=options)
    
    base = 'https://www.goodreads.com/book/show/'
    
    url = base + item
    # Wait for page to load
    wait = WebDriverWait(driver, 30)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'Text__title1')))
        
    # Convert the page source to a Beautiful Soup object
    soup = BeautifulSoup(driver.page_source, 'html.parser')
        
    print(driver.execute_script('return document.readyState'))  # Should print 'complete' 
    # Save to variable
    title_tag = soup.find('h1', attrs={'class': 'Text Text__title1'}) #.text
    # Looking at the h1 tag aria.        
    title = title_tag['aria-label'].split(': ')[1]
    print(f'URL:              {url}')
    print(f'Book Title:       {title}')

In [ ]:
df = pd.read_csv('book_urls.txt', header=None)

In [ ]:
df = df.rename(columns={0:'links'})

In [ ]:
df.head()

In [ ]:
base = 'https://www.goodreads.com/book/show/'

df = pd.read_csv('2000_book_ids.txt', header= None, names = ['book_tag'])

practice = base +  df['book_tag'][0]
practice

In [ ]:
response = requests.get(practice)

soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
soup

In [ ]:
collect = pd.DataFrame(columns = ['title','summary','year_published','author','review_count',
                                  'number_of_ratings','length','genre','rating','list','reviews'])

In [ ]:
soup.find('h1', attrs={'class': 'Text Text__title1'})

title_tag = soup.find('h1', attrs={'class': 'Text Text__title1'})

In [ ]:
title = title_tag['aria-label'].split(': ')[1]

In [ ]:
title

In [ ]:
collect.loc[0, 'title'] = title

In [ ]:
title

In [ ]:
def get_bad_books_links(df):
    # Check if links.txt exists, read the last link as the starter_link
    if os.path.exists("links.txt"):
        with open("links.txt", "r") as file:
            lines = file.readlines()
            starter_link = lines[-1].strip()
    else:
        starter_link = "https://www.goodreads.com/book/show/3450744-nudge"

    # Add new column called 'link' to df
    df['link'] = ""

    driver = webdriver.Chrome()

    # Open a text file for writing
    with open("links.txt", "a") as file:

        # If row_index.txt exists, read the last index and start from the next row
        if os.path.exists("row_index.txt"):
            with open("row_index.txt", "r") as index_file:
                last_index = int(index_file.read().strip())
                start_index = last_index + 1
        else:
            start_index = 0

        for index, row in df.iloc[start_index:].iterrows():
            # loading initial webpage
            driver.get(starter_link)

            # current row content to use in query
            title = row['Title']
            author = row['Author']

            try:
                # add wait for page to finish loading
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Header"]/div[2]/div[2]/section/form/input[1]')))

                # search GoodReads for "title author"
                search_bar = driver.find_element_by_xpath('//*[@id="Header"]/div[2]/div[2]/section/form/input[1]')
                search_bar.send_keys(title + " " + author)
                search_bar.submit()

                # add wait for page to finish loading
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'bookTitle')))

                # extract 1st search result link from page
                links = driver.find_element_by_class_name('bookTitle')
                row_value = links.get_attribute('href')

                # append value to new column called 'link'
                df.at[index, 'link'] = row_value

                # Update starter_link
                starter_link = row_value

                # Write the row_value to the text file
                file.write(f"{row_value}\n")

            except:
                # If no search results or timeout, continue to the next row
                continue

            finally:
                # Save current row index to row_index.txt
                with open("row_index.txt", "w") as index_file:
                    index_file.write(str(index))

    driver.quit()
    return df

In [ ]:
def get_book_info(filename, col):
    
    df = pd.read_csv(filename, header= None, names = [col])

    base = 'https://www.goodreads.com/book/show/'

    collect = pd.DataFrame(columns = ['title','summary','year_published','author','review_count',
                                  'number_of_ratings','length','genre','rating','reviews'])

    options = Options()
    options.headless = True
    service = Service('/path/to/chromedriver') # replace with the path to your chromedriver executable
    driver = webdriver.Chrome(service=service, options=options)

    counter = 0

    for i, item in enumerate(df['book_tag'][375:]):
    
        url = base + item
        driver.get(url)
        # Wait for the page to load
        wait = WebDriverWait(driver, 30)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'Text__title1')))

        # Convert the page source to a BeautifulSoup object
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        print(driver.execute_script('return document.readyState'))  # should print 'complete' 
        print(f'URL:              {url}')
        # print(f'Request Response: {response.status_code}')
        print(f'Request Number:   {counter+1}')
    
        #--------title-------------------------------------------
    
        # save to Variable
        title_tag = soup.find('h1', attrs={'class': 'Text Text__title1'}) #.text
        # looking at the h1 tag aria.        
        title = title_tag['aria-label'].split(': ')[1]
        print(f'Book Title: {title}')
    
        #----author-----------------------------------------------
    
        # find where author is stored
        soup.find('span', class_ = "ContributorLink__name")
        # save the text to a variable
        auth = soup.find('span', class_ = "ContributorLink__name").text
        print(f'Book Author: {auth}')
    
        #----rating-----------------------------------------------
    
        # 
        rating = soup.find_all('div', class_= 'RatingStatistics__rating')[0].text
        print(f'Overall Rating: {rating}')
    
        #----genre------------------------------------------------
    
        try:
            genre = soup.find('a', {'class': 'Button Button--tag-inline Button--small'}).get_text()
            print(f'Genre: {genre}')
        except:
            continue
    
        #----rating and review counts------------------------------
    
        stats = soup.find_all('div', class_ = "RatingStatistics__meta")
    
        rating_span = soup.find('span', {'data-testid': 'ratingsCount'})
        review_span = soup.find('span', {'data-testid': 'reviewsCount'})
    
        rating_text = rating_span.text.strip()
        review_text = review_span.text.strip()
    
        rating_text = re.sub('[^0-9]', '', rating_text)
        review_text = re.sub('[^0-9]', '', review_text)
        print(f'Number of Ratings: {rating_text}')
        print(f'Number of Reviews: {review_text}')
         
        #----summary------------------------------
    
        summary = soup.find_all('span', class_ = "Formatted")[0].text
           
        #----# of pages------------------------------
    
        page = soup.find('p').text
        page = re.sub('[^0-9]', '', page)
        print(f'Number of Pages: {page}')
          
        #--------year--------------------------
    
        year = (soup.find_all('p')[1].text)
        year = year[-4:]
        print(f'Year Published: {year}')
    
        #--------reviews--------------------------

        # reviews = soup.find_all('section',class_ = "ReviewText__content")
          
        # ls_reviews = []
        # for i in range (0,len (reviews)):
        #    rev = soup.find_all('section',class_ = "ReviewText__content")[i].text
        #    ls_reviews.append(rev)
    
        print('---------------------------------------------')
        
        collect.loc[counter, ['title','summary','year_published','author','review_count','number_of_ratings',
                          'length','genre','rating']] = title, summary, year, auth, review_text, rating_text, page, genre, rating  
        counter += 1 
    

### subsetting each part of the loop into a function